In [2]:
!rm hawaii.sqlite

In [3]:
import pandas as pd
from sqlalchemy import create_engine, ForeignKey,func,inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float,Date
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session



In [4]:
Base = declarative_base()
engine = create_engine("sqlite:///hawaii.sqlite")

In [5]:
#creating classes

class Measurement(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    #text for date
    date = Column(Integer)
    prcp = Column(Float)
    tobs = Column(Float)
  



class Station(Base):
    __tablename__ = 'stations'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation= Column(Float)
    


In [6]:
Base.metadata.create_all(engine)

In [7]:
def populate_table(engine, table, csvfile):
    """Populates a table from a Pandas DataFrame."""
    # connect to the database
    conn = engine.connect()
    
    # Load the CSV file into a pandas dataframe 
    df_of_data_to_insert = pd.read_csv(csvfile)
    
    # Orient='records' creates a list of data to write
    # http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
    data = df_of_data_to_insert.to_dict(orient='records')
    
    # Optional: Delete all rows in the table 
    #conn.execute(table.delete())
    
    # Insert the dataframe into the database in one bulk insert
    conn.execute(table.insert(), data)
    
# Call the function to insert the data for each table
populate_table(engine, Station.__table__, 'clean_hawaii_stations.csv')
populate_table(engine, Measurement.__table__,'clean_hawaii_measurements.csv')

In [ ]:
#An alternate method
#csv_measurements.to_sql('measurements',engine,if_exists='append',index_label='id')
#csv_stations.to_sql('stations',engine, if_exists='replace',index_label = 'id')

In [8]:
#checking the tables
inspector = inspect(engine)
print(inspector.get_table_names())

['measurements', 'stations']


In [9]:
#checking the tables
engine.execute('SELECT * FROM measurements LIMIT 5').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]

In [10]:
engine.execute('SELECT * FROM stations LIMIT 5').fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]